In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [23]:
#Read csv and turn into dataframe 
datapath = 'data/Brainwaves_sales_2022-12-12.csv' 
df = pd.read_csv(datapath, encoding = "utf-8", skiprows=[0]) 

#Change date column to datetime format 
df.date = pd.to_datetime(arg=df.date, errors='raise', format="%m/%d/%y %I:%M%p").dt.date
# df.date = df.date.dt.strftime('%Y-%m-%d')

#clean column data 
df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_')

#get rid of unnecessary columns 
df = df.drop(columns=['paid_to','ship_from_country_name', 'package', 'option', \
    'item_url','catalog_number', 'upc', 'isrc', 'buyer_name', 'buyer_email', \
        'buyer_phone', 'buyer_note', 'ship_to_name', 'ship_to_street', 'ship_to_street_2', \
            'ship_to_city', 'ship_to_state', 'seller_tax', 'ship_to_zip', 'ship_to_country', \
                'ship_to_country_code', 'buyer_country_code', 'buyer_country_name', 'ship_date', \
                    'discount_code', 'ship_notes'],\
                    axis=0)
df = df.fillna(0)

df

,date,item_type,item_name,artist,currency,item_price,quantity,sub_total,marketplace_tax,shipping,...,bandcamp_transaction_id,paypal_transaction_id,assessed_revenue_share,collected_revenue_share,balance_of_revenue_share_(EUR),balance_of_revenue_share_(CHF),change_to_payout_balance,payout_balance_(EUR),payout_balance_(CHF),net_amount
0,2018-09-14,album,Mosfet EP,Tout Casser,CHF,1.1,1.0,1.1,0.00,0.0,...,3842591446,0,0.17,0.17,0.0,0.0,0.77,0.00,0.77,0.77
1,2018-09-14,album,Mosfet EP,Tout Casser,CHF,2.0,1.0,2.0,0.00,0.0,...,2372949730,0,0.30,0.30,0.0,0.0,1.49,0.00,2.26,1.49
2,2018-09-14,album,Mosfet EP,Tout Casser,CHF,5.0,1.0,5.0,0.00,0.0,...,441260497,0,0.75,0.75,0.0,0.0,3.86,0.00,6.12,3.86
3,2018-09-17,album,Mosfet EP,Tout Casser,CHF,1.0,1.0,1.0,0.00,0.0,...,657318030,0,0.15,0.15,0.0,0.0,0.70,0.00,6.82,0.70
4,2018-09-20,track,From Sand To Silicon,Tout Casser,CHF,0.5,1.0,0.5,0.00,0.0,...,3363304614,0,0.08,0.08,0.0,0.0,0.30,0.00,7.12,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,2022-12-10,track,Luk fukin gud pls,Tout Casser,EUR,1.5,1.0,1.5,0.00,0.0,...,2099835016,0,0.23,0.23,0.0,0.0,1.14,-7.91,0.00,1.14
7199,2022-12-10,track,Situationship,Hermeth,EUR,1.2,1.0,1.2,0.20,0.0,...,3845260784,0,0.18,0.18,0.0,0.0,0.89,-7.02,0.00,0.89
7200,2022-12-10,track,So,Hermeth,EUR,1.0,1.0,1.0,0.23,0.0,...,1239413740,0,0.15,0.15,0.0,0.0,0.73,-6.29,0.00,0.73
7201,2022-12-11,payout,0,0,EUR,0.0,0.0,0.0,0.00,0.0,...,t670203990,3G619159GD714174S,0.00,0.00,0.0,0.0,-8.77,-15.06,0.00,0.00


In [3]:
payouts = df[df['item_type'] == 'payout']
payouts = payouts.drop(columns=['item_name', 'artist', 'item_price', 'quantity', 'sub_total', 'marketplace_tax', 'shipping',\
    'fee_type','assessed_revenue_share', 'collected_revenue_share', 'balance_of_revenue_share_(EUR)', 'balance_of_revenue_share_(CHF)',\
        'net_amount'])
payouts.amount_you_received.sum()

In [22]:
#Read csv and turn into dataframe 
paypal_url = 'data/Paypal.csv'
paypal_df = pd.read_csv(paypal_url)  

#Turn date into datetime formats 
paypal_df.Date = pd.to_datetime(arg=paypal_df.Date, errors='raise', format="%d/%m/%Y")

#Clean column data
paypal_df.columns = paypal_df.columns\
    .str.replace(' ', '_')\
    .str.replace('/', '_')
paypal_df.columns = [column.lower() for column in paypal_df.columns]

#Remove unnecessary columns 
paypal_df = paypal_df.drop(['time', 'timezone', 'from_email_address','to_email_address','shipping_address','item_id','insurance_amount',\
    'option_1_name','option_1_value','option_2_name','option_2_value','reference_txn_id','invoice_number','custom_number',\
        'receipt_id','address_line_1','address_line_2_district_neighborhood','town_city','state_province_region_county_territory_prefecture_republic',\
            'zip_postal_code','country','contact_phone_number','note','country_code'], axis=1)

#Seperate debit and credit columns 
paypal_debit = paypal_df[paypal_df['balance_impact'] == 'Debit']
paypal_credit = paypal_df[paypal_df['balance_impact'] == 'Credit']

,date,name,type,status,currency,gross,fee,net,transaction_id,address_status,item_title,shipping_and_handling_amount,sales_tax,quantity,balance,subject,balance_impact
0,2018-01-06,NaN,General Card Deposit,Completed,CHF,"48,35","0,00","48,35",4LL52338D30597426,NaN,Discogs Order #163907-5792,"0,00","0,00",1.0,"48,35",Discogs Order #163907-5792,Credit
1,2018-01-06,NaN,General Currency Conversion,Completed,CHF,"-48,35","0,00","-48,35",0F805853W71723432,NaN,Discogs Order #163907-5792,"0,00","0,00",1.0,"0,00",Discogs Order #163907-5792,Debit
2,2018-01-06,NaN,General Currency Conversion,Completed,EUR,"39,55","0,00","39,55",30453837S3595630Y,NaN,Discogs Order #163907-5792,"0,00","0,00",1.0,"39,55",Discogs Order #163907-5792,Credit
3,2018-01-06,wolvie records,Website Payment,Completed,EUR,"-39,55","0,00","-39,55",0D543655D5124844P,Confirmed,Discogs Order #163907-5792,"0,00","0,00",1.0,"0,00",Discogs Order #163907-5792,Debit
4,2018-03-05,"Bandcamp, Inc.",Express Checkout Payment,Completed,EUR,"-17,00","0,00","-17,00",2E310818CE696930A,Confirmed,"12"" Black Vinyl -WRECKS016 - Dawl + Sween ""RIS...","6,00","0,00",1.0,"-17,00","12"" Black Vinyl -WRECKS016 - Dawl + Sween ""RIS...",Debit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,2022-12-09,"Bandcamp, an Epic Company",Mass Pay Payment,Completed,EUR,"7,98","0,00","7,98",41F3361436977961G,Non-Confirmed,NaN,NaN,NaN,NaN,"396,11",You've got money from Bandcamp,Credit
3987,2022-12-09,Google Payment Corporation,PreApproved Payment Bill User Payment,Completed,CHF,"-4,50","0,00","-4,50",5B289816UG1987842,Non-Confirmed,Parasite,"0,00","0,00",1.0,"-4,50",Parasite,Debit
3988,2022-12-09,NaN,General Currency Conversion,Completed,EUR,"-4,75","0,00","-4,75",14370646EK780150V,NaN,Parasite,"0,00","0,00",1.0,"391,36",Parasite,Debit
3989,2022-12-09,NaN,General Currency Conversion,Completed,CHF,"4,50","0,00","4,50",7HX75350SE6736230,NaN,Parasite,"0,00","0,00",1.0,"0,00",Parasite,Credit
